# **03. 네이버 Geocoding API 사용 위·경도 구하기**

**사용 후 Client ID와 Client Secret값을 삭제했으므로 아래 코드는 작동하지 않습니다**

In [1]:
import requests
import pandas as pd

In [2]:
hospital = pd.read_excel("전국병의원_202306.xlsx")
hospital = hospital.loc[hospital['시도코드명'].isin(['서울', '경기', '인천']), ['요양기관명', '종별코드명', '시도코드명', '주소']]

hospital['종별코드명'] = hospital['종별코드명'].replace(['상급종합'], '3차')
hospital['종별코드명'] = hospital['종별코드명'].replace(['종합병원', '병원', '한방병원', '치과병원'], '2차')
hospital['종별코드명'] = hospital['종별코드명'].replace(['의원', '치과의원', '한의원', '보건소', '보건지소', '보건진료소', '보건의료원'], '1차')

hospital = hospital.loc[hospital['종별코드명'].isin(['1차', '2차', '3차'])]

In [3]:
client_id = "(개인정보)"
client_secret = "(개인정보)"

addresses = hospital['주소'].apply(lambda x: x.split(",")[0]).to_list()

location_list = []
for address in addresses: 
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        }

    res = requests.get(url, headers=headers)
    address_data = res.json().get('addresses')  
    # .get(): 딕셔너리에서 주어진 키에 해당하는 값을 가져오고 만약 키가 존재하지 않는 경우에도 오류 없이 None을 반환한다

    if address_data:
        lng = address_data[0]['x']
        lat = address_data[0]['y']
        location = (float(lat), float(lng))
    else:
        location = (0.0, 0.0)
        
    location_list.append(location)

hospital['location'] = location_list

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 1-4: ordinal not in range(256)

In [ ]:
hospital[hospital['location'] == (0,0)]

,요양기관명,종별코드명,시도코드명,주소,location


In [ ]:
hospital.to_csv("hospital.csv", index=False)

In [ ]:
subway = pd.read_csv("subway.csv")

In [ ]:
client_id = "(개인정보)"
client_secret = "(개인정보)"

addresses = subway['역사도로명주소'].to_list()

location_list = []
for address in addresses: 
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        }

    res = requests.get(url, headers=headers)
    address_data = res.json().get('addresses')  

    if address_data:
        lng = address_data[0]['x']
        lat = address_data[0]['y']
        location = (float(lat), float(lng))
    else:
        location = (0.0, 0.0)
        
    location_list.append(location)

subway['fixed_location'] = location_list

In [ ]:
errored = subway['fixed_location'] == (0.0, 0.0)
subway.loc[errored, 'fixed_location'] = subway.loc[errored, 'location']

array([(37.5716094, 126.7360234), (37.566412, 126.7426159),
       (37.5536441, 126.7450563), (37.5450619, 126.7386231),
       (37.54329, 126.7280806), (37.5382559, 126.722932),
       '(37.530271, 126.722548)', (37.5168167, 126.7215307),
       (37.5082254, 126.7205148), (37.4984159, 126.7222643),
       (37.4904473, 126.7234864), (37.4851943, 126.7179525),
       (37.4779397, 126.7100277), (37.4698162, 126.7085323),
       (37.4574194, 126.7023421), (37.4494459, 126.7011634),
       (37.441905, 126.6996942), (37.4347557, 126.6983669),
       (37.4271249, 126.698957), (37.4179545, 126.6938545),
       (37.4121079, 126.6876275), (37.4049076, 126.6807471),
       (37.3982309, 126.6736974), (37.3883567, 126.6621868),
       (37.382397, 126.6565188), (37.3781203, 126.6453829),
       (37.3860462, 126.6393889), (37.3930099, 126.6347311),
       (37.4003041, 126.6301544), (37.4073268, 126.6257459),
       (37.5949667, 126.627967), (37.5952639, 126.6426556),
       (37.601675, 126.6564585),

In [ ]:
subway.to_csv("fixed_subway.csv", encoding='UTF-8', index=False)